In [23]:
import pandas as pd 
import re
from glob import glob

In [24]:
2254 + 2080 + 2041 + 1988

8363

In [26]:
sapro_tables = [(pd.read_excel(path)
         .assign(Year='20' + re.search('\d\d', path).group())
) for path in glob('../data/intermediate_usm_data/sapro/SAPRO Annual Reports/SAPRO_*_all.xlsx')
                + ['../data/intermediate_usm_data/sapro/SAPRO Annual Reports/SAPRO_charts_army_14.xlsx']
]

def clean_colname(col):
    return (col.replace('\r', ' ')
                  .replace('Administra- tive Discharge Type', 'Administrative Discharge Type')
                  .replace('Confinemen t', 'Confinement')
                  .replace('Incident Location', 'Location')
                  .replace('Quarter Dispositio n Complete d', 'Quarter Disposition Completed')
                  .replace('applicable.', 'applicable')
                  .replace('Acession ?', 'Acession?')
                  .replace('Restrictio n', 'Restriction')
                  .replace('Investi- gation', 'Investigation')
                  .replace('Victim Grade', 'Victim Pay Grade')
                  .replace('Subject Grade', 'Subject Pay Grade')
                  .replace('Subject Affiliation', 'Subject Service')
                  )

cleaned_tables = [table.rename(
    columns={col:clean_colname(col) for col in table.columns}
) for table in sapro_tables]

sapro_full = pd.concat(cleaned_tables).reset_index(drop=True)
sapro_full = sapro_full.applymap(lambda x: x.replace('\r', ' ') if isinstance(x, basestring) else x)

In [31]:
def sapro_get_court_martial_outcome(row):
    if pd.notnull(row):
        row = row.lower()
        if row !='null':
            if ('dismissed' in row) or ('acquit' in row):
                if ('subsequent' in row) or ('prior' in row):
                    outcome = 'no court-martial'
                else:
                    outcome = 'not guilty'
            elif ('in lieu' in row) or ('article 15' in row) or ('art 15' in row):
                outcome = 'no court-martial'
            else:
                outcome = 'guilty'
            return outcome
    return 'no court-martial'

sapro_full['Outcome'] = sapro_full['Court Case or Article 15 Outcome'].apply(
    sapro_get_court_martial_outcome
)

In [36]:
import random
import numpy as np
import copy

In [41]:
index_c = copy.copy(list(sapro_full.index))
np.random.shuffle(index_c)

In [45]:
sapro_shuffelled = sapro_full.loc[index_c].reset_index(drop=True).copy()

In [80]:
chunk_size = len(sapro_shuffelled) / 20

In [82]:
chunks = range(0, len(sapro_shuffelled), chunk_size)[:-1] + [8365]
chunks = zip(chunks[:-1], chunks[1:])

In [87]:
class_data = []
for start, stop in chunks:
    class_data.append(sapro_shuffelled.iloc[start:stop])

In [ ]:
sapro_shuffelled['Sub']

In [99]:
# Order by Acquittal (so people see "Acquittals" first)
# Order by Rank (so people read the convictions of higher-ranks first)
# Order by word count (so the lengthier cases are first ... those are more likely to be meaty)

class_data[0]['Case Synopsis'].fillna('').apply(lambda x: len(x))

0        0
1        0
2        0
3        0
4        0
5        0
6      508
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
      ... 
388      0
389      0
390      0
391      0
392      0
393      0
394      0
395      0
396      0
397      0
398      0
399      0
400      0
401     56
402      0
403      0
404      0
405      0
406    172
407      0
408      0
409      0
410      0
411    765
412      0
413      0
414      0
415      0
416      0
417      0
Name: Case Synopsis, Length: 418, dtype: int64